In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2
import logging
import sys
import numpy as np
import scipy.special as special
import scipy.integrate as integrate

In this notebook we will copy some of the stuff from `eko` and will try to benchmark it and make it faster. Most of the time is spent in the `_run_singlet` function, so logically we will need to copy and call this function.

In [2]:
from eko.constants import Constants
from eko import interpolation
from eko.dglap import _get_xgrid
from eko import alpha_s
from eko import mellin
from eko import splitting_functions_LO as sf_LO

cc = Constants()

# Grid definition
xgrid_low = interpolation.get_xgrid_linear_at_log(5,1e-7,0.1)
xgrid_mid = interpolation.get_xgrid_linear_at_id(5,0.1,1.0)
xgrid_high = 1.0-interpolation.get_xgrid_linear_at_log(1,1e-3,1.0 - 0.9)
xgrid = np.unique(np.concatenate((xgrid_low,xgrid_mid,xgrid_high)))

# Target grid
toy_xgrid = np.array([1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,.1,.3,.5,.7,.9])
toy_xgrid = np.array([1e-7, 1e-2])
nfFF = 4

# Delta t
a_s = alpha_s.alpha_s_generator(0.35, 2.0, nfFF, "analytic")
a0 = a_s(0, 2)
a1 = a_s(0, 1e4)
delta_t = np.log(1.0/a1) - np.log(1.0/a0)

polynom_rank = 4
beta0 = alpha_s.beta_0(nfFF, cc.CA, cc.CF, cc.TF)

In [3]:
from eko.dglap import _run_singlet

# Long taking function
inverse_mellin = mellin.inverse_mellin_transform

ret = {"operators" : {}, "operator_errors" : {}}
is_log_interpolation = True
basis_function_coeffs = interpolation.get_Lagrange_basis_functions(xgrid, polynom_rank)
def fake_run_singlet():
    targetgrid = toy_xgrid
    targetgrid_size = len(targetgrid)
    nf = nfFF

    # prepare
    def get_kernels_s(j,lnx):
        """return siglet integration kernels"""
        current_coeff = basis_function_coeffs[j] #areas dictionary from
        # which are passed a list of:
        #                 (logxmin, logxmax, list of coefficients)
        fN = interpolation.evaluate_Lagrange_basis_function_log_N
        def get_ker(k,l):
            def ker(N):
                """singlet integration kernel"""
                l_p,l_m,e_p,e_m = sf_LO.get_Eigensystem_gamma_singlet_0(N,nf,cc.CA,cc.CF)
                ln_p = - delta_t * l_p  / beta0
                ln_m = - delta_t * l_m  / beta0
                interpoln = fN(N,current_coeff,lnx)
                return (e_p[k][l] * np.exp(ln_p) + e_m[k][l] * np.exp(ln_m)) * interpoln
            return ker

        return get_ker(0,0), get_ker(0,1), get_ker(1,0), get_ker(1,1)

    # perform
    xgrid_size = len(xgrid)
    void = np.zeros((targetgrid_size, xgrid_size))
    op_s_qq = np.copy(void)
    op_s_qg = np.copy(void)
    op_s_gq = np.copy(void)
    op_s_gg = np.copy(void)
    op_s_qq_err = np.copy(void)
    op_s_qg_err = np.copy(void)
    op_s_gq_err = np.copy(void)
    op_s_gg_err = np.copy(void)

    for k,xk in enumerate(targetgrid):
        if False and xk < 1e-3:
            cut = 0.1
            gamma = 2.0
        else:
            cut = 1e-2
            gamma = 1.0
        path,jac = mellin.get_path_Cauchy_tan(gamma,1.0)
        for j in range(xgrid_size):
            # iterate all matrix elements
            ker_qq,ker_qg,ker_gq,ker_gg = get_kernels_s(j,np.log(xk))
            for ker,op,op_err in [
                    (ker_qq,op_s_qq,op_s_qq_err),(ker_qg,op_s_qg,op_s_qg_err),
                    (ker_gq,op_s_gq,op_s_gq_err),(ker_gg,op_s_gg,op_s_gg_err)
                ]:
                res = inverse_mellin(ker, path, jac, cut)
                op[k, j] = res[0]
                op_err[k, j] = res[1]

    # insert operators
    ret["operators"]["S_qq"] = op_s_qq
    ret["operators"]["S_qg"] = op_s_qg
    ret["operators"]["S_gq"] = op_s_gq
    ret["operators"]["S_gg"] = op_s_gg
    ret["operator_errors"]["S_qq"] = op_s_qq_err
    ret["operator_errors"]["S_qg"] = op_s_qg_err
    ret["operator_errors"]["S_gq"] = op_s_gq_err
    ret["operator_errors"]["S_gg"] = op_s_gg_err
    return ret

In [4]:
mydict = {
    "PTO": 0,
    'alphas': 0.35,
    'Qref': np.sqrt(2),
    'Q0': np.sqrt(2),
    'NfFF': 4,

    "xgrid_type": "custom",
    "xgrid_custom": xgrid,
    "xgrid" : xgrid,
    "xgrid_polynom_rank": polynom_rank,
    "xgrid_interpolation": "log",
    "targetgrid": toy_xgrid,
    "Q2grid": [1e4],
    "operators" : {},
    "operator_errors" : {}
}
%time reference_result = _run_singlet(mydict, cc, delta_t, True, basis_function_coeffs, mydict)

/home/jumax9/Academic_Workspace/N3PDF/eko/src/eko/mellin.py:53: IntegrationWarning: The maximum number of subdivisions (100) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  result = integrate.quad(integrand, 0.5, 1.0 - cut,epsabs=1e-11,epsrel=1e-11,limit=100)
/home/jumax9/Academic_Workspace/N3PDF/eko/src/eko/mellin.py:53: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result = integrate.quad(integrand, 0.5, 1.0 - cut,epsabs=1e-11,epsrel=1e-11,limit=100)
/home/jumax9/Academic_Workspace/N3PDF/eko/src/eko/mellin.py:53: Integr

CPU times: user 1min 13s, sys: 22.7 ms, total: 1min 13s
Wall time: 1min 13s


In [5]:
# Compare the results of _run_singlet and its fake version and check they are the same
#%lprun -T lprof0 -f inverse_mellin_transform result_fake = fake_run_singlet()
#print(open('lprof0', 'r').read())
%time result_fake = fake_run_singlet()

/home/jumax9/Academic_Workspace/N3PDF/eko/src/eko/mellin.py:53: IntegrationWarning: The maximum number of subdivisions (100) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  result = integrate.quad(integrand, 0.5, 1.0 - cut,epsabs=1e-11,epsrel=1e-11,limit=100)
/home/jumax9/Academic_Workspace/N3PDF/eko/src/eko/mellin.py:53: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result = integrate.quad(integrand, 0.5, 1.0 - cut,epsabs=1e-11,epsrel=1e-11,limit=100)
/home/jumax9/Academic_Workspace/N3PDF/eko/src/eko/mellin.py:53: Integr

CPU times: user 1min 10s, sys: 6.68 ms, total: 1min 10s
Wall time: 1min 10s


Of course, we already knew that the problematic function was
`res = inverse_mellin(ker, path, jac, cut)` so if we can make this function faster, the whole code will become much faster. Let's try running it for one single example.

In [6]:
# First let us get the path and the jacobian
xk = 0.1
j = 1
# These two are numba functions so nothing to do here
path,jac = mellin.get_path_Cauchy_tan(1.0,1.0)
import numba as nb

# prepare
def get_kernels_s(j,lnx):
    """return siglet integration kernels"""
    current_coeff = basis_function_coeffs[j]
    fN = interpolation.evaluate_Lagrange_basis_function_log_N
    CA = cc.CA
    cf = cc.CF
    def get_ker(k,l):
        #@nb.jit(forceobj=True)
        def ker(N):
            """singlet integration kernel"""
            l_p,l_m,e_p,e_m = sf_LO.get_Eigensystem_gamma_singlet_0(N,nfFF,CA,cf)
            ln_p = - delta_t * l_p  / beta0
            ln_m = - delta_t * l_m  / beta0
            interpoln = fN(N,current_coeff,lnx)
            return (e_p[k][l] * np.exp(ln_p) + e_m[k][l] * np.exp(ln_m)) * interpoln
        return ker
    return get_ker(0,0), get_ker(0,1), get_ker(1,0), get_ker(1,1)
ker_qq, ker_qg, ker_gq, ker_gg = get_kernels_s(j,np.log(xk))

cut = 1e-2
fun = ker_gg
    
#@nb.jit(forceobj=True)
def integrand(u):
    N = path(u)
    prefactor = complex(0.0, -1.0 / 2.0 / np.pi)
    result = 2.0 * np.real(prefactor * fun(N) * jac(u))
    return result

%time result = integrate.quad(integrand, 0.5, 1.0 - cut,epsrel=1e-6,limit=100)

CPU times: user 559 ms, sys: 0 ns, total: 559 ms
Wall time: 559 ms


<timed exec>:1: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.


In [13]:
# Proof of concept, uncomment numba decorator to check the speed up
import numba as nb

def fun(x):
    y = np.sin(x)
    for i in range(100):
        y += np.exp(-i)/np.cos(y)
    return y

nbfun = nb.njit(fun)
%time res = integrate.quad(nbfun, 0.5, 1.0)

print(res)

CPU times: user 71.7 ms, sys: 2 µs, total: 71.7 ms
Wall time: 71 ms
(0.22317357630324017, 0.25631938458284664)


<timed exec>:1: IntegrationWarning: The integral is probably divergent, or slowly convergent.


In [8]:
basis_function_coeffs[0]

{'polynom_number': 0,
 'areas': [{'lower_index': 0,
   'reference_indices': (0, 4),
   'coeffs': array([ 1.03372284e+00, -3.37566323e+05,  3.37893542e+09, -1.06747844e+12,
           1.03372284e+13]),
   'xmin': 1e-07,
   'xmax': 3.162277660168379e-06},
  {'lower_index': 1,
   'reference_indices': (0, 4),
   'coeffs': array([ 1.03372284e+00, -3.37566323e+05,  3.37893542e+09, -1.06747844e+12,
           1.03372284e+13]),
   'xmin': 3.162277660168379e-06,
   'xmax': 0.0001},
  {'lower_index': 2,
   'reference_indices': (0, 4),
   'coeffs': array([ 1.03372284e+00, -3.37566323e+05,  3.37893542e+09, -1.06747844e+12,
           1.03372284e+13]),
   'xmin': 0.0001,
   'xmax': 0.0031622776601683794}]}

In [9]:
basis_function_coeffs[9]

{'polynom_number': 9,
 'areas': [{'lower_index': 7,
   'reference_indices': (5, 9),
   'coeffs': array([  2024.95061728, -14552.16278006,  36656.24142661, -38759.94513032,
           14631.91586648]),
   'xmin': 0.775,
   'xmax': 0.999},
  {'lower_index': 8,
   'reference_indices': (5, 9),
   'coeffs': array([  2024.95061728, -14552.16278006,  36656.24142661, -38759.94513032,
           14631.91586648]),
   'xmin': 0.999,
   'xmax': 1.0}]}

### Numba tries

In [20]:
def test_function(n):
    result = np.identity(2)*4
    return result

nb_test = nb.njit(test_function)
nb_test(4)

array([[4., 0.],
       [0., 4.]])

In [45]:
import numba as nb
import quadpy

def fun(x, extra_num = 4.4888):
    y = np.sin(x)
    for i in range(100):
        y += np.exp(-i)/np.cos(y)
    return y*extra_num

nbfun = nb.njit(fun)
%time res = quadpy.quad(nbfun, 0.5, 1.0)

print(res)

KeyboardInterrupt: 

(22.31735763032409, 25.63193845828505)
